In [1]:
import sys
sys.path.insert(0, '..')
import numpy as np
import os
import yaml
import sequenceanalyzer as sa
import sequence_generator as sg
import partition as pt
import partitionset as ps
import sequence_generator as sg
import dmarkov as dm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans as k
import save_plot as sp
import eigenvectorcalcs as eig
import moore
from scipy.special import lambertw
from scipy.stats.mstats import gmean
# sys.path.insert(0, 'Code')

# path = 'logistic_map'
# L = 10000000

# path = 'ternary_even_shift'
# L = 10000000

path = '10dbq1'
L = 50000000

### Parameters

In [2]:
K = 5
Drange = [3,4,5,6,7,8]
kmeans_types = ['standard', 'kl', 'weighted']
machine_types = ['dmarkov'] + kmeans_types

In [3]:
def dist(a, b):
    return np.linalg.norm(np.array(a) - np.array(b), axis=1)

def kdist(a, b):
    eps = 1e-5
    a = np.array(a, dtype='float64')
    b = np.array(b, dtype='float64')
    a[a == 0] = eps
    b[b == 0] = eps
    kl = (a*np.log(a/b) + b*np.log(b/a))/2
    
    return np.sum(kl, axis = 1)
    
def kl_divergence(a, b):
    eps = 1e-10
    a = np.array(a, dtype='float64')
    b = np.array(b, dtype='float64')
    a[a == 0] = eps
    b[b == 0] = eps
    kl = a*np.log(a/b) + b - a

    return np.sum(kl)

def find_centroid(data, alpha = 1e-5, debug = False):
    eps = 1e-5

    data[data == 0] = eps
    g = gmean(data)
    g[g == 0] = eps
    a = np.mean(data, axis = 0)

    c0 = a
    c = c0 + 1
    lambda_ = - kl_divergence(c0, g)

    while np.linalg.norm(abs(c - c0)) > 1e-5:
        c0 = c.copy()
        for i, ci in enumerate(c):
            den = lambertw(a[i]*(np.exp(lambda_ +1))/g[i])
            c[i] = a[i]/den
        lambda_ = - kl_divergence(c, g)
        if debug:
            print(f'new: {c}, previous: {c0}')

    return c

def get_initial_centroids(samples = [], K = 5):
    i = 0
    centroids = []

    for sample in samples:
        sample = list(sample)
        if sample not in centroids:
            centroids.append(sample)
            i += 1
        if i == K:
            return np.array(centroids)
    return centroids

# K-Means using KL-divergence as metric
def kkmeans(matrix, k, centroids = []):
    if not centroids:
        centroids = get_initial_centroids(matrix, k)
    current_centroids = np.array(centroids)
    previous_centroids = np.zeros(current_centroids.shape)

    error = dist(current_centroids, previous_centroids)
    nearest_clusters = np.zeros(len(matrix))
    
    while sum(error) != 0:
        for i in range(len(matrix)):
            distances = kdist(matrix[i], current_centroids)
            nearest_clusters[i] = np.argmin(distances)

            previous_centroids = current_centroids.copy()

        for i in range(k):
#             current_centroids[i] = np.mean(matrix[np.where(nearest_clusters == i)], axis=0)
             current_centroids[i] = find_centroid(matrix[np.where(nearest_clusters == i)])

        error = dist(current_centroids, previous_centroids)
    return (nearest_clusters, current_centroids)

# Standard K-Means
def kmeans(matrix, k, centroids = []):
    if not centroids:
        centroids = get_initial_centroids(matrix, k)
    current_centroids = np.array(centroids)
    previous_centroids = np.zeros(current_centroids.shape)

    error = dist(current_centroids, previous_centroids)
    nearest_clusters = np.zeros(len(matrix))
    
    while sum(error) != 0:
        for i in range(len(matrix)):
            distances = dist(matrix[i], current_centroids)
            nearest_clusters[i] = np.argmin(distances)

            previous_centroids = current_centroids.copy()

        for i in range(k):
            current_centroids[i] = np.mean(matrix[np.where(nearest_clusters == i)], axis=0)
#              current_centroids[i] = find_centroid(matrix[np.where(nearest_clusters == i)])

        error = dist(current_centroids, previous_centroids)
    return (nearest_clusters, current_centroids)

# K-Means weighted
def kmeans_w(matrix, k, weights, centroids = []):
    if not centroids:
        centroids = get_initial_centroids(matrix, k)
    current_centroids = np.array(centroids)
    previous_centroids = np.zeros(current_centroids.shape)

    error = dist(current_centroids, previous_centroids)
    nearest_clusters = np.zeros(len(matrix))
    
    while sum(error) != 0:
        for i in range(len(matrix)):
            distances = dist(matrix[i], current_centroids)
            nearest_clusters[i] = np.argmin(distances)

            previous_centroids = current_centroids.copy()

        for i in range(k):
            current_centroids[i] = np.average(matrix[np.where(nearest_clusters == i)], axis=0, weights=weights[np.where(nearest_clusters == i)])
#              current_centroids[i] = find_centroid(matrix[np.where(nearest_clusters == i)])

        error = dist(current_centroids, previous_centroids)
    return (nearest_clusters, current_centroids)

def get_center(features, target):
    target_set = set(target)
    return np.array([np.mean(features[np.where(target == i)], 0) for i in range(len(target_set))])

# closest_cluster, kmeans_centers = kmeans(morphs, K, initial_centroids)

In [5]:
# def logistic_map(x0 = 0.5, r = 3.75):
#     x = [x0]
#     s = ''
#     for i in range(L):
#         x.append(r*x[i]*(1-x[i]))
#         if x[i] <= 0.67:
#              s += '0'
#         elif x[i] <= 0.79:
#              s += '1'
#         else:
#              s += '2'
#     return s

# # os.makedirs('logistic_map/sequences')
# # os.makedirs('logistic_map/machine')
# # os.makedirs('logistic_map/probabilities/conditional')
# s = logistic_map()

# with open('logistic_map/sequences/len_10000000.yaml', 'w') as f:
#     yaml.dump(s, f)

### Load sequence with length L from path

In [4]:
with open(f'{path}/sequences/len_{L}.yaml', 'r') as f:
    original_sequence = yaml.load(f)

### Compute probabilities, conditional probabilities, alphabet and machines

In [11]:
prob, alph = sa.calc_probs(original_sequence, 12)
prob_cond = sa.calc_cond_probs(prob, alph, 12)

with open(f'{path}/probabilities/len_{L}.yaml', 'w') as f:
    yaml.dump(prob, f)
with open(f'{path}/probabilities/conditional/len_{L}.yaml', 'w') as f:
    yaml.dump(prob_cond, f)
with open(f'{path}/probabilities/alphabet/len_{L}.yaml', 'w') as f:
    yaml.dump(alph, f)

NameError: name 'seq' is not defined

### Load probabilities, conditional probabilities, alphabet and machines

In [63]:
with open(f'{path}/probabilities/len_{L}.yaml', 'r') as f:
    prob = yaml.load(f)
with open(f'{path}/probabilities/conditional/len_{L}.yaml', 'r') as f:
    prob_cond = yaml.load(f)
with open(f'{path}/probabilities/alphabet/len_{L}.yaml', 'r') as f:
    alph = yaml.load(f)

In [64]:
machines = {}
centers = {}

for mtype in machine_types:
    machines[mtype] = {}
    centers[mtype] = {}
    
markov = machines['dmarkov']
# normal_m = {}
# weighted_m = {}
# kl_m = {}

# normal_centers = {}
# weighted_centers = {}
# kl_centers = {}

for D in Drange:
    markov[D] = dm.DMarkov(prob_cond, D, alph, prob)
    print(f'Number of states for D = {D}: {len(markov[D].states)}')

    with open(f'{path}/machine/dmarkov_D{D}_L{L}.yaml', 'w') as f:
        yaml.dump(markov[D], f)

Number of states for D = 3: 8
Number of states for D = 4: 16
Number of states for D = 5: 32
Number of states for D = 6: 64
Number of states for D = 7: 128
Number of states for D = 8: 256


## Normal K-Means

In [65]:
for D in Drange:
    machine = markov[D]
    # K-Means
    idx = dict((s.name, machine.states.index(s)) for s in machine.states)
    morphs = []
    all_oedges = [state.outedges for state in machine.states]
    state_probs = np.array([state.state_prob for state in machine.states])

    for oedges in all_oedges:
        curr_morph = [0] * len(machine.index_labels)
        for oedge in oedges:
            label = oedge[0]
            curr_morph[machine.index_labels[label]] = oedge[-1]
        morphs.append(curr_morph)
    morphs = np.array(morphs)
    
    for ktype in kmeans_types:
        # data = morphs.T
        # data.shape
        if ktype == 'standard':
            closest_cluster, centers[ktype][D] = kmeans(morphs, K)
        elif ktype == 'kl':
            closest_cluster, centers[ktype][D] = kkmeans(morphs, K)
        elif ktype == 'weighted':
            closest_cluster, centers[ktype][D] = kmeans_w(morphs, K, state_probs)
            
        #MOORE
        clusters = [[] for i in closest_cluster]
        # print(f"Clusterization check")
        for i in range(len(morphs)):
            cluster_index = int(closest_cluster[i])
            # print(f"\tCenter: {kmeans.cluster_centers_[state_idx]}, Outedge: {machine.states[i].outedges}")
            clusters[cluster_index].append(machine.states[i])
        # Fix empty clusters problem
        clusters = [c for c in clusters if c]
        # Split cluster if two or more states have differente outedges

        new_clusters = []

        for c in clusters:
            new_clusters_dict = dict()
            for st in c:
                key = ''.join([oedge[0] for oedge in st.outedges])
                if key in new_clusters_dict:
                    new_clusters_dict[key].append(st)
                else:
                    new_clusters_dict[key] = [st]
            for new_c in new_clusters_dict.values():
                new_clusters.append(new_c)

        initial_pt = []

        for p in new_clusters:
            partition = pt.Partition()
            for state in p:
                partition.add_to_partition(state)
            initial_pt.append(partition)

        initial_pt = ps.PartitionSet(initial_pt)
        final_pt = moore.moore_by_parts(machine, initial_pt, n_iter = -1)
        
        machines[ktype][D] = final_pt.redefine_partition(markov[D])

        with open(f'{path}/machine/{ktype}_D{D}_L{L}.yaml', 'w') as f:
            yaml.dump(machines[ktype][D], f)

Moore finished with 2 iterations
Moore finished with 2 iterations
Moore finished with 2 iterations
Moore finished with 3 iterations
Moore finished with 3 iterations
Moore finished with 4 iterations
Moore finished with 5 iterations


/home/sid/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: ComplexWarning: Casting complex values to real discards the imaginary part


Moore finished with 5 iterations
Moore finished with 5 iterations
Moore finished with 5 iterations
Moore finished with 5 iterations
Moore finished with 6 iterations
Moore finished with 5 iterations
Moore finished with 5 iterations
Moore finished with 6 iterations
Moore finished with 5 iterations
Moore finished with 5 iterations
Moore finished with 6 iterations


In [66]:
for D in Drange:
    alpha = 1
    for ktype in kmeans_types:
        plt.plot([a[0] for a in centers[ktype][D]], [a[1] for a in centers[ktype][D]], '*', markersize = 14, alpha = alpha, label = ktype)
        alpha -= 0.4
    plt.legend()
    plt.savefig(f'{path}/figs/centroids_comparison_{D}.png')
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [67]:
original_entropy = sa.calc_cond_entropy(prob, prob_cond, 10)

for D in Drange:
    for mtype in machine_types:
        sequence, occup_vector = sg.generate_sequence_and_occup_vector(machines[mtype][D], L)
        
        new_prob, new_alph = sa.calc_probs(sequence,10)
        new_cond_prob = sa.calc_cond_probs(new_prob, new_alph, 10)
        
        e = sa.calc_cond_entropy(new_prob, new_cond_prob, 10)
        kl = sa.calc_kldivergence(prob, new_prob, 10)
        
        print(f'D = {D}, K = {K}, Kmeans = {mtype}\r\n\tEntropies: {e}\r\n\tKL: {kl}')
        
        with open(f'{path}/sequences/{mtype}_D{D}_L{L}.yaml', 'w') as f:
            yaml.dump(sequence, f)
        with open(f'{path}/metrics/cond_entropies/{mtype}_D{D}_L{L}.yaml', 'w') as f:
            yaml.dump(e, f)
        with open(f'{path}/metrics/kl_divergences/{mtype}_D{D}_L{L}.yaml', 'w') as f:
            yaml.dump(kl, f)
        with open(f'{path}/metrics/occup_vector/{mtype}_D{D}_L{L}.yaml', 'w') as f:
            yaml.dump(occup_vector, f)

Calculating conditional entropy for sequence at: 
L = 10
*****************
Sequence: 
Conditional entropy calculated!
*****************
Calculating probabilities for words with length 11 ...
Calculating probabilities for words with length 10 ...
Calculating probabilities for words with length 9 ...
Calculating probabilities for words with length 8 ...
Calculating probabilities for words with length 7 ...
Calculating probabilities for words with length 6 ...
Calculating probabilities for words with length 5 ...
Calculating probabilities for words with length 4 ...
Calculating probabilities for words with length 3 ...
Calculating probabilities for words with length 2 ...
Calculating subsequence conditional probabilities
L = 10
Calculating conditional probabilities of subsequences of length: 1
Calculating conditional probabilities of subsequences of length: 2
Calculating conditional probabilities of subsequences of length: 3
Calculating conditional probabilities of subsequences of length:

### Load machines and metrics for plotting

In [68]:
metrics = ['kl_divergences', 'cond_entropies', 'occup_vector', 'n_states']
results = {}

for metric in metrics:
    results[metric] = {}
    for mtype in machine_types:
        results[metric][mtype] = []
        
original_entropy = sa.calc_cond_entropy(prob, prob_cond, 10)

for D in Drange:
    for mtype in machine_types:
        with open(f'{path}/machine/{mtype}_D{D}_L{L}.yaml', 'r') as f:
            machines[mtype][D] = yaml.load(f)
            
        for metric in metrics:
            if metric == 'n_states':
                results['n_states'][mtype].append(len(machines[mtype][D].states))
            else:
                with open(f'{path}/metrics/{metric}/{mtype}_D{D}_L{L}.yaml', 'r') as f:
                    if metric == 'cond_entropies':
                        results[metric][mtype].append(yaml.load(f)[-1])
                    else:
                        results[metric][mtype].append(yaml.load(f))
                        

Calculating conditional entropy for sequence at: 
L = 10
*****************
Sequence: 
Conditional entropy calculated!
*****************


In [69]:
for metric in metrics[0:2]:
    for mtype in machine_types:
        plt.plot(results['n_states'][mtype], results[metric][mtype], '+-', label = mtype)
    if metric == 'cond_entropies':
        plt.axhline(y=original_entropy[-1], color='k', alpha = 0.4, linestyle='-', label='Original')
    print('breakline')
    plt.legend()
    plt.xscale('log')
    plt.savefig(f'{path}/figs/{metric} for D: {Drange[0]}, {Drange[-1]}')
    plt.clf()

breakline
breakline


<Figure size 432x288 with 0 Axes>

### TODO:

- [OK] Plotar divergência de Kullback-Leibler

- [OK] Plotar entropia da sequência original no gráfico de entropias condicionais

- Tentar implementar KL weighted

### Application

- [OK] Dividir sequência original em pedaços de tamanho n (255) 

- [OK] Calcular número de 1's (erros - m, valor não precisa ser muito grande (probabilidade vai ficar muito pequena)) 

- [OK] Fazer mesma coisa para sequências geradas pela máquina

- [OK] Plotar P(m, n) x m - Probabilidade de m erros em palavras de tamanho n por número de erros

In [5]:
from itertools import islice

def chunk(it, size):
    it = iter(it)
    return np.array(list(iter(lambda: tuple(islice(it, size)), ())))

def error_counter(seq, n, m):
    words = chunk(seq, n)
    # checks if sequence isn't multiple of n and delete last word if True
    if len(words[-1]) != n:
        words = words[0:-1]
    
    prob_distribution = np.zeros(n)
    
    for w in words:
        w = np.array(list(map(int, w)))
        count = w.sum()
        prob_distribution[count] += 1
    
    return prob_distribution/np.sum(prob_distribution)

In [6]:
prob_distribution = {}

original_distribution = error_counter(original_sequence, 255, 10)

for mtype in machine_types:
    prob_distribution[mtype] = []
    print(f"type is {mtype}")
    for D in Drange:
        print(f"D is {D}")
        with open(f'{path}/sequences/{mtype}_D{D}_L{L}.yaml', 'r') as f:
            sequence = yaml.load(f)
        prob_distribution[mtype].append(error_counter(sequence, 255, 10))

In [18]:
m = 35
for mtype in machine_types:
    plt.plot(original_distribution[0:m+1], label = 'original')
    for i, p in enumerate(prob_distribution[mtype]):
        if i > 3:
            plt.plot(p[0:m+1], label = Drange[i])
    plt.yscale('log')
    plt.legend()
    plt.ylim(0.000001, 0.2)
    plt.savefig(f'{path}/figs/{mtype}_p_255.eps')
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [21]:
for mtype in machine_types:
    print(f"type is {mtype}")
    with open(f'{path}/metrics/prob_distributions/{mtype}_L{L}.yaml', 'w') as f:
        yaml.dump(prob_distribution[mtype], f)

type is dmarkov
type is standard
type is kl
type is weighted


In [10]:
prob_distribution

{'dmarkov': [array([2.58366568e-02, 6.12817348e-02, 9.06016993e-02, 1.09711441e-01,
         1.16142556e-01, 1.12302247e-01, 1.03106927e-01, 8.76283928e-02,
         7.46335642e-02, 5.80687278e-02, 4.46709983e-02, 3.45780761e-02,
         2.52195555e-02, 1.75746387e-02, 1.26429278e-02, 8.60371893e-03,
         5.89561297e-03, 3.97800875e-03, 2.62140577e-03, 1.60650353e-03,
         1.20870266e-03, 7.95601750e-04, 4.84501066e-04, 3.00900662e-04,
         1.93800426e-04, 1.17300258e-04, 5.61001234e-05, 6.63001459e-05,
         4.08000898e-05, 1.02000224e-05, 5.10001122e-06, 1.02000224e-05,
         0.00000000e+00, 0.00000000e+00, 5.10001122e-06, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00,